# Make bookkeeping with Holograms


- work with Weakly_2022_09
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/06/16


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/atmospec 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_09/conda/envs/lsst-scipipe-2.0.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/rapid_analysis 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g64fc59b30a+e1dba0a6f6 	w_latest w_2022_09 current setup


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [4]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [5]:
import lsst.daf.butler as dafButler

In [6]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

In [7]:
collection = "u/dagoret/first_test2"
datasetRefs1 = registry.queryDatasets(datasetType='postISRCCD', collections=collection, where= "instrument='LATISS'")
datasetRefs2 = registry.queryDatasets(datasetType='calexp', collections=collection, where= "instrument='LATISS'")

## List of Exposures

In [8]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs','group_name' ,'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','postISRCCD','calexp'])

In [9]:
df_exposure.astype({"id":str,
                    "obs_id":str,
                    "day_obs": str,
                    "group_name":str,
                    "seq_num":'int32',
                    'type':str,
                    "target":str,
                    "filter":str,
                    "zenith_angle":'float',
                    "expos":'float',
                    "ra":'float',
                    "dec":'float',
                    "skyangle":'float',
                    "postISRCCD":bool,
                    "calexp":bool
                   }              
                  )

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp


In [10]:
#df_exposure = pd.DataFrame( {"id":'int64',
#                             "obs_id":str,
#                             "day_obs": str,
#                             "seq_num":'int32',
#                             "type":str,
#                             "target":str,
#                             "filter":str,
#                             "zenith_angle":'float32',
#                             "expos":'float32',
#                             "ra":'float32',
#                             "dec":'float32',
#                             "skyangle":'float32'})                  

In [11]:
for count, info in enumerate(registry.queryDimensionRecords('exposure')):
    
    flag_postisrccd = False
    flag_calexp = False
    
    for count, ref in enumerate(datasetRefs1):    
        if ref.dataId["exposure"]== info.id:
            flag_postisrccd = True
            break
    for count, ref in enumerate(datasetRefs2):    
        if ref.dataId["exposure"]== info.id:
            flag_calexp = True
            break        
            
    
    
    
    df_exposure.loc[count] = [str(info.id), str(info.obs_id), str(info.day_obs), str(info.group_name),int(info.seq_num), str(info.observation_type), str(info.target_name), str(info.physical_filter), info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,flag_postisrccd,flag_calexp]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t group_name:          ",info.group_name)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [12]:
df_exposure.dtypes

id               object
obs_id           object
day_obs          object
group_name       object
seq_num           int64
type             object
target           object
filter           object
zenith_angle    float64
expos           float64
ra              float64
dec             float64
skyangle        float64
postISRCCD         bool
calexp             bool
dtype: object

In [13]:
df_exposure

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp
2163,2022062900553,AT_O_20220629_000553,20220629,2022-06-30T06:15:46.324,553,acq,HD 139763,SDSSr~empty,62.957457,5.0,235.144339,-18.048923,86.808155,False,False
97,2021070700320,AT_O_20210707_000320,20210707,2021-07-08T04:49:21.820,320,science,HD 160617,empty~holo4_003,20.545791,20.0,265.639181,-40.262824,-1.402388,True,False
95,2021070700319,AT_O_20210707_000319,20210707,2021-07-08T04:49:21.820,319,science,HD 160617,empty~holo4_003,20.454183,20.0,265.636042,-40.264335,-1.682716,True,False
99,2021070700318,AT_O_20210707_000318,20210707,2021-07-08T04:49:21.820,318,science,HD 160617,empty~holo4_003,20.362633,20.0,265.632998,-40.265829,-1.964402,True,False
98,2021070700321,AT_O_20210707_000321,20210707,2021-07-08T04:49:21.820,321,science,HD 160617,empty~holo4_003,20.640626,20.0,265.642182,-40.261426,-1.114358,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,2022060900838,AT_O_20220609_000838,20220609,2022-06-10T08:52:06.482,838,science,spec:HD185975,BG40~holo4_003,57.194196,30.0,307.708009,-87.506860,105.702010,True,False
1357,2022060900839,AT_O_20220609_000839,20220609,2022-06-10T08:52:06.482,839,science,spec:HD185975,FELH0600~holo4_003,57.196255,30.0,307.707897,-87.506868,105.701922,True,False
1367,2022060900840,AT_O_20220609_000840,20220609,2022-06-10T08:52:06.482,840,science,spec:HD185975,FELH0600~holo4_003,57.198265,30.0,307.708264,-87.506854,105.701541,True,False
1368,2022060900841,AT_O_20220609_000841,20220609,2022-06-10T08:52:06.482,841,science,spec:HD185975,FELH0600~holo4_003,57.200339,30.0,307.708109,-87.506873,105.701644,True,False


## Selection of science

In [14]:
df_science = df_exposure[df_exposure.type == 'science']

In [15]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_1802/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [16]:
df_science.tail(60)

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp
2094,2022060900727,AT_O_20220609_000727,20220609,2022-06-10T07:03:20.848,727,science,spec:HD205905,empty~holo4_003,25.468183,30.0,324.793859,-27.258692,346.438740,True,False
2096,2022060900728,AT_O_20220609_000728,20220609,2022-06-10T07:03:20.848,728,science,spec:HD205905,empty~holo4_003,25.312973,30.0,324.793835,-27.258698,346.438648,True,False
2099,2022060900729,AT_O_20220609_000729,20220609,2022-06-10T07:03:20.848,729,science,spec:HD205905,empty~holo4_003,25.157745,30.0,324.793791,-27.258701,346.438682,True,False
2103,2022060900730,AT_O_20220609_000730,20220609,2022-06-10T07:03:20.848,730,science,spec:HD205905,empty~holo4_003,25.002597,30.0,324.793834,-27.258687,346.435105,True,False
2067,2022060900731,AT_O_20220609_000731,20220609,2022-06-10T07:03:20.848,731,science,spec:HD205905,BG40~holo4_003,24.836547,30.0,324.793803,-27.258706,346.438635,True,False
2077,2022060900732,AT_O_20220609_000732,20220609,2022-06-10T07:03:20.848,732,science,spec:HD205905,BG40~holo4_003,24.681380,30.0,324.793823,-27.258686,346.438678,True,False
2080,2022060900733,AT_O_20220609_000733,20220609,2022-06-10T07:03:20.848,733,science,spec:HD205905,BG40~holo4_003,24.526203,30.0,324.793840,-27.258693,346.438776,True,False
2085,2022060900734,AT_O_20220609_000734,20220609,2022-06-10T07:03:20.848,734,science,spec:HD205905,BG40~holo4_003,24.371006,30.0,324.793831,-27.258697,346.438737,True,False
1293,2022060900735,AT_O_20220609_000735,20220609,2022-06-10T07:03:20.848,735,science,spec:HD205905,FELH0600~holo4_003,24.208584,30.0,324.793811,-27.258695,346.438688,True,False
1295,2022060900736,AT_O_20220609_000736,20220609,2022-06-10T07:03:20.848,736,science,spec:HD205905,FELH0600~holo4_003,24.053380,30.0,324.793792,-27.258704,346.438666,True,False


# Generate Book Keeping files

In [17]:
writer = pd.ExcelWriter('BookKeepingAuxtelHologram_raw.xlsx', engine='xlsxwriter')
current_page = 0

In [18]:
ListOfDates = df_science.day_obs.unique()

In [19]:
ListOfDates

array(['20210707', '20211102', '20211103', '20211104', '20220215',
       '20220216', '20220316', '20220317', '20220607', '20220608',
       '20220609'], dtype=object)

In [20]:
for date_sel in ListOfDates:
    
    # get for that date
    df_science_selected = df_science[df_science.day_obs == date_sel ]
    N = len(df_science_selected)
    
    list_of_filters = df_science_selected['filter'].unique()
    
    selected_filters = []
    # Select hologram
    for filt in list_of_filters:
        #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
        flag_sel = (filt.find('holo4') != -1) 
        #flag_sel = (filt.find('ronchi170') != -1) 
        if flag_sel:
            selected_filters.append(filt) 
    Nfilt = len(selected_filters)
    # sort by filter name
    if(Nfilt>0):
        selected_filters = np.array(selected_filters)
        df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
        df_science_selected.to_excel(writer, sheet_name=str(date_sel))
        


/tmp/ipykernel_1802/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
/tmp/ipykernel_1802/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
/tmp/ipykernel_1802/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_va

In [21]:
writer.save()